In [1]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub

2023-11-05 11:14:54.540489: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-05 11:14:54.721758: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-05 11:14:54.721816: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-05 11:14:54.722294: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-05 11:14:54.806093: I tensorflow/core/platform/cpu_feature_g

In [2]:
# import tensorflow_hub as hub
# import tensorflow as tf
# #Elmo
# elmo = hub.load("https://tfhub.dev/google/elmo/3").signatures["default"]
# input_tensor = np.array(["my birthday is the best day ","today is a good day","sdf "])
# embeddings_tensor = elmo(tf.constant(input_tensor))["elmo"]  #, signature="default", as_dict=True)
# print(embeddings_tensor.shape)

In [2]:
import numpy as np
import pandas as pd
import json
# import torch
import nltk
import random
import re                                 
import string
from cleantext import clean            
from nltk.corpus import stopwords     
from nltk.stem import PorterStemmer       
from nltk.tokenize import TweetTokenizer 
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone',
        'time', 'date', 'number'],
    # terms that will be annotated
    # annotate={"hashtag", "allcaps", "elongated", "repeated",
    #     'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    segmenter="twitter", 
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for  
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[emoticons]
)
class Dataset(object):
    def __init__(self, root_dir, max_seq_length, mode = "train", transform=None):
        self.root_dir = root_dir
        self.mode = mode
        self.max_seq_length = max_seq_length
        self.text_processor = text_processor
        self.data = self.read_data(root_dir)
        if self.mode != "test":
            self.labels = self.data.labels.tolist()
        self.classes = list(pd.read_csv("./HW1_dataset/sample_submission.csv").columns[1:])
        self.tokenizer =  TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
        self.tweet_length,self.tweet_token = self.preprocess(self.data.tweet)
    def preprocess(self, tweets):
        def filter_words(x):
            x = re.sub(r'@[A-Za-z0-9_]+', '', x)
            # x = re.sub(r'[@#][^\s]*\s*|\s*[@#][^\s]*$', '', x)
            # x = re.sub(r'RT[\s]+', '', x)
            # x = re.sub(r"@([A-Za-z0-9_]{4,15})", r"@ <user>", x)
            # x = re.sub(r'\$', '', x)
            # x = re.sub(r'\+', '', x)
            # x = re.sub(r'\|', '', x)
            # x = re.sub(r'\.\.\.', '', x)
            x = x.replace("cant","can't")
            x = x.replace("wont","won't")
            x = x.replace("Im","I'm")
            x = x.replace("Ill","I'll")
            x = x.replace("Ive","I have")
            x = x.replace("I've","I have")
            x = x.replace("youre","you're")
            x = x.replace("theyre","they're")
            x = x.replace("thats","that's")
            x = x.replace("whats","what's")
            x = x.replace("dont","don't")
            x = x.replace("didnt","didn't")
            x = x.replace("doesnt","doesn't")
            x = x.replace(".","") 
            x = x.replace("nvm","never mind")
            x = clean(x, no_emoji=True, no_urls =True, no_numbers=True, replace_with_url="", replace_with_number="<NUMBER>")
            
            return x
        tweets = tweets.apply(lambda x : filter_words(x))
        tweet_token = tweets.apply(lambda x : self.text_processor.pre_process_doc(x))
        tweet_length = tweet_token.apply(lambda x : len(x))
        return tweet_length.to_numpy(), tweet_token.to_numpy()
    def padding(self,x):
        tweets = []
        for tweet in x:
            if len(tweet)>=self.max_seq_length:
                tweets.append(tweet[:self.max_seq_length])
            else:
                tweets.append(tweet + [""]*(self.max_seq_length-len(tweet)))
        return tweets
    def encode_label(self, labels):
        targets = np.zeros((len(labels),12))
        
        for i,label in enumerate(labels):
            for l in label:
                idx = self.classes.index(l)
                targets[i,idx] = 1
        return targets
    def read_data(self, root_dir):
        with open(root_dir, "r") as f:
            data = json.load(f)
        df =  pd.DataFrame(data)
        if self.mode != "test":
            df["labels"] = df.labels.apply(lambda x : list(x.keys()))
        return df 
    def get_data(self):
        self.tweet_token = self.padding(self.tweet_token)
        if self.mode != "test":
            return [self.tweet_token,self.encode_label(self.labels)]
        else:
            return [self.tweet_token]

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
/home/samuel/.conda/envs/tf/lib/python3.9/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


/home/samuel/.conda/envs/tf/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [3]:
train_data_path = "./HW1_dataset/train.json"
val_data_path = "./HW1_dataset/val.json"
test_data_path = "./HW1_dataset/test.json"
train_dataset = Dataset(train_data_path, 55, mode = "train", transform=None)
val_dataset = Dataset(val_data_path, 55,mode = "val", transform=None)
test_dataset = Dataset(test_data_path, 55,mode = "test", transform=None)
train_data = train_dataset.get_data()
val_data = val_dataset.get_data()
test_data = test_dataset.get_data()

In [4]:
from tqdm import tqdm
elmo = hub.load("https://tfhub.dev/google/elmo/3").signatures["default"]
seq_length = 55
embedding_dim = 1024
def generate_embeddings(data):
    embeddings = np.zeros((len(data[0]),seq_length,embedding_dim))
    for i in tqdm(range(len(data[0]))):
        try:
            data_tweet = tf.constant(data[0][i])
            embeddings[i] = tf.squeeze(elmo(data_tweet)["elmo"])
        except Exception as e:
            print(e)
            print("index:",i)
            print(data_tweet.shape)
            print(data_tweet)
    return embeddings

2023-11-05 11:15:36.632206: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-05 11:15:36.657537: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
train_embeddings = generate_embeddings(train_data)
val_embeddings = generate_embeddings(val_data)
test_embeddings = generate_embeddings(test_data)

 77%|███████▋  | 5346/6956 [09:35<02:37, 10.23it/s]

could not broadcast input array from shape (55,3,1024) into shape (55,1024)
index: 5344
(55,)
tf.Tensor(
[b'now' b'should' b'we' b'trust' b'pfizer' b'in' b'injecting' b'our'
 b'bodies' b'( ? )' b'but' b'here' b'is' b'the' b'nigerian' b'meningitis'
 b'scandal' b'pfizer' b'was' b'involved' b'in' b'a' b'<number>' b'year'
 b'battle' b'that' b'is' b'now' b'one' b'of' b'the' b'biggest' b'scandals'
 b'in' b'the' b'pharmaceutical' b'world' b'note' b'it' b'was' b'regarding'
 b'a' b'drug' b'not' b'vaccine' b'https' b'://' b'tco' b'/' b'brvlcuv7cn'
 b'' b'' b'' b'' b''], shape=(55,), dtype=string)


 78%|███████▊  | 5458/6956 [09:45<02:20, 10.63it/s]

In [11]:
train_data[1][5344] = np.zeros(12)
np.save("./elmo_embedding/new_preprocess_noAnotate_elmo_X_train.npy",train_embeddings)
np.save("./elmo_embedding/new_preprocess_noAnotate_elmo_X_val.npy",val_embeddings)
np.save("./elmo_embedding/new_preprocess_noAnotate_elmo_X_test.npy",test_embeddings)
np.save("./elmo_embedding/new_preprocess_noAnotate_elmo_y_train.npy",train_data[1])
np.save("./elmo_embedding/new_preprocess_noAnotate_elmo_y_val.npy",val_data[1])
